In [ ]:
from neo4j import GraphDatabase
import json
from pprint import pprint

import os
from dotenv import load_dotenv

from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.retrievers import Text2CypherRetriever, H


load_dotenv()

True

## 1. Connect to Neo4j

In [3]:
# Connection details
NEO4J_URL = os.getenv("NEO4J_URL")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

# Create driver
driver = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Test connection
with driver.session() as session:
    result = session.run("RETURN 1 as test")
    print("✓ Connection successful!")
    print(f"Test query result: {result.single()['test']}")

✓ Connection successful!
Test query result: 1


## 2. Get Database Schema

In [4]:
def get_schema(driver):
    """Fetch and display the database schema."""
    with driver.session() as session:
        # Get node labels
        labels_result = session.run("CALL db.labels()")
        labels = [record["label"] for record in labels_result]
        
        # Get relationship types
        rels_result = session.run("CALL db.relationshipTypes()")
        rel_types = [record["relationshipType"] for record in rels_result]
        
        print("=== Node Labels ===")
        for label in labels:
            print(f"  - {label}")
        
        print(f"\nTotal: {len(labels)} labels")
        
        print("\n=== Relationship Types ===")
        for rel in rel_types:
            print(f"  - {rel}")
        
        print(f"\nTotal: {len(rel_types)} relationship types")

get_schema(driver)

=== Node Labels ===
  - GraphNode
  - module
  - screen
  - form
  - form_field
  - session
  - function
  - database_table
  - TabOverview
  - TabProcessingFlow
  - TabImpact
  - tab
  - ProcessFlow
  - form_group

Total: 14 labels

=== Relationship Types ===
  - uses_table
  - DESCRIBES
  - DESCRIBES_FLOW
  - IMPACTS
  - AFFECTS
  - BELONGS_TO
  - HAS_GROUP
  - CONTAINS_FIELD

Total: 8 relationship types


## 3. Get Database Statistics

In [5]:
def get_stats(driver):
    """Get basic statistics about the database."""
    with driver.session() as session:
        # Count nodes
        node_count = session.run("MATCH (n) RETURN count(n) as count").single()["count"]
        
        # Count relationships
        rel_count = session.run("MATCH ()-[r]->() RETURN count(r) as count").single()["count"]
        
        # Count by label
        label_counts = session.run("""
            CALL db.labels() YIELD label
            CALL {
                WITH label
                MATCH (n)
                WHERE label IN labels(n)
                RETURN count(n) as count
            }
            RETURN label, count
            ORDER BY count DESC
        """).data()
        
        print("=== Database Statistics ===")
        print(f"Total Nodes: {node_count}")
        print(f"Total Relationships: {rel_count}")
        print("\n=== Nodes by Label ===")
        for item in label_counts:
            print(f"  {item['label']}: {item['count']}")

get_stats(driver)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (label) { ... }} {position: line: 3, column: 13, offset: 54} for query: '\n            CALL db.labels() YIELD label\n            CALL {\n                WITH label\n                MATCH (n)\n                WHERE label IN labels(n)\n                RETURN count(n) as count\n            }\n            RETURN label, count\n            ORDER BY count DESC\n        '


=== Database Statistics ===
Total Nodes: 230
Total Relationships: 251

=== Nodes by Label ===
  GraphNode: 230
  form_field: 88
  database_table: 83
  TabImpact: 19
  ProcessFlow: 6
  form_group: 6
  screen: 5
  session: 5
  function: 4
  tab: 4
  form: 3
  TabOverview: 3
  TabProcessingFlow: 3
  module: 1


## 4. Sample Node Properties

In [6]:
def show_sample_nodes(driver, label="GraphNode", limit=3):
    """Show sample nodes with their properties."""
    with driver.session() as session:
        result = session.run(f"""
            MATCH (n:{label})
            RETURN n
            LIMIT $limit
        """, limit=limit)
        
        print(f"=== Sample {label} Nodes ===")
        for i, record in enumerate(result, 1):
            node = record["n"]
            print(f"\nNode {i}:")
            print(f"  Labels: {list(node.labels)}")
            print(f"  Properties:")
            for key, value in dict(node).items():
                # Truncate long values
                if isinstance(value, str) and len(value) > 100:
                    value = value[:100] + "..."
                print(f"    {key}: {value}")

show_sample_nodes(driver)

=== Sample GraphNode Nodes ===

Node 1:
  Labels: ['GraphNode', 'session']
  Properties:
    lifecycle: Set when coming from sales selection; Used in sales integration flow
    uid: session:visitInfo
    scope: session
    session_key: visitInfo
    name: Visit Information Session
    description: Stores sales visit information when contract is created from sales selection screen - enables sales ...
    parent_module: module:simple
    source_file: ctc-data-en/simple/yuusyou-kihon/screen-flow-en.md
    stored_data: VisitInfoVO object containing sales visit information including selected items and visit context

Node 2:
  Labels: ['GraphNode', 'database_table']
  Properties:
    uid: database_table:t_keiyaku
    name: Keiyaku Transaction Table
    description: Main contract table - System core
    table_name: t_keiyaku
    source_file: ctc-data-en/simple/yuusyou-kihon/functions/init-screen/function-overview-en.md
    table_type: transaction

Node 3:
  Labels: ['GraphNode', 'database_tab

## 5. Test Text-to-Cypher Schema Generation

In [7]:
def get_detailed_schema(driver):
    """Get detailed schema suitable for Text2Cypher."""
    with driver.session() as session:
        # Get node type properties
        try:
            node_schema = session.run("""
                CALL db.schema.nodeTypeProperties()
                YIELD nodeType, propertyName, propertyTypes, mandatory
                RETURN nodeType, propertyName, propertyTypes, mandatory
            """).data()
        except:
            # Fallback for older Neo4j versions
            node_schema = []
        
        # Get relationship type properties
        try:
            rel_schema = session.run("""
                CALL db.schema.relTypeProperties()
                YIELD relType, propertyName, propertyTypes
                RETURN relType, propertyName, propertyTypes
            """).data()
        except:
            rel_schema = []
        
        print("=== Detailed Schema for Text2Cypher ===")
        print("\nNode Types and Properties:")
        if node_schema:
            for item in node_schema:
                mandatory = item.get('mandatory', False)
                marker = " *" if mandatory else ""
                print(f"  {item['nodeType']}.{item['propertyName']}: {item['propertyTypes']}{marker}")
        else:
            print("  (Schema introspection not available)")
        
        print("\nRelationship Types and Properties:")
        if rel_schema:
            for item in rel_schema:
                print(f"  {item['relType']}.{item['propertyName']}: {item['propertyTypes']}")
        else:
            print("  (Schema introspection not available)")

get_detailed_schema(driver)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Procedure.ProcedureWarning} {category: GENERIC} {title: The query used a procedure that generated a warning.} {description: The query used a procedure that generated a warning. (The field `propertyTypes` will change output format in the next major version.)} {position: line: 2, column: 17, offset: 17} for query: '\n                CALL db.schema.nodeTypeProperties()\n                YIELD nodeType, propertyName, propertyTypes, mandatory\n                RETURN nodeType, propertyName, propertyTypes, mandatory\n            '
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Procedure.ProcedureWarning} {category: GENERIC} {title: The query used a procedure that generated a warning.} {description: The query used a procedure that generated a warning. (The field `propertyTypes` will change output format in the next major version.)} {position: line: 2, column: 17, 

=== Detailed Schema for Text2Cypher ===

Node Types and Properties:
  :`GraphNode`:`module`.name: ['String'] *
  :`GraphNode`:`module`.uid: ['String'] *
  :`GraphNode`:`module`.parent_module: ['String'] *
  :`GraphNode`:`module`.module_name: ['String'] *
  :`GraphNode`:`module`.description: ['String'] *
  :`GraphNode`:`module`.tab_based_interface: ['Boolean'] *
  :`GraphNode`:`module`.key_features: ['String'] *
  :`GraphNode`:`module`.source_file: ['String'] *
  :`GraphNode`:`module`.main_jsp: ['String'] *
  :`GraphNode`:`module`.documentation_structure: ['String'] *
  :`GraphNode`:`module`.main_screens: ['String'] *
  :`GraphNode`:`screen`.name: ['String'] *
  :`GraphNode`:`screen`.uid: ['String'] *
  :`GraphNode`:`screen`.parent_module: ['String'] *
  :`GraphNode`:`screen`.description: ['String'] *
  :`GraphNode`:`screen`.tab_code: ['String']
  :`GraphNode`:`screen`.has_tabs: ['Boolean']
  :`GraphNode`:`screen`.main_jsp: ['String'] *
  :`GraphNode`:`screen`.title: ['String'] *
  :`Gr

## 6. Run Custom Cypher Query

In [8]:
def run_query(driver, query, params=None):
    """Run a custom Cypher query."""
    with driver.session() as session:
        result = session.run(query, params or {})
        data = result.data()
        print(f"Query returned {len(data)} results\n")
        for i, record in enumerate(data, 1):
            print(f"Result {i}:")
            pprint(record)
            print()
        return data

# Example: Find nodes with specific uid
query = """
MATCH (n:GraphNode)
RETURN n.uid as uid, n.name as name, labels(n) as labels
LIMIT 5
"""

run_query(driver, query)

Query returned 5 results

Result 1:
{'labels': ['GraphNode', 'session'],
 'name': 'Visit Information Session',
 'uid': 'session:visitInfo'}

Result 2:
{'labels': ['GraphNode', 'database_table'],
 'name': 'Keiyaku Transaction Table',
 'uid': 'database_table:t_keiyaku'}

Result 3:
{'labels': ['GraphNode', 'database_table'],
 'name': 'Keiyaku Kaikeitani Kankei Transaction Table',
 'uid': 'database_table:t_keiyaku_kaikeitani_kankei'}

Result 4:
{'labels': ['GraphNode', 'database_table'],
 'name': 'Anken Transaction Table',
 'uid': 'database_table:t_anken'}

Result 5:
{'labels': ['GraphNode', 'database_table'],
 'name': 'Tochi Transaction Table',
 'uid': 'database_table:t_tochi'}



[{'uid': 'session:visitInfo',
  'name': 'Visit Information Session',
  'labels': ['GraphNode', 'session']},
 {'uid': 'database_table:t_keiyaku',
  'name': 'Keiyaku Transaction Table',
  'labels': ['GraphNode', 'database_table']},
 {'uid': 'database_table:t_keiyaku_kaikeitani_kankei',
  'name': 'Keiyaku Kaikeitani Kankei Transaction Table',
  'labels': ['GraphNode', 'database_table']},
 {'uid': 'database_table:t_anken',
  'name': 'Anken Transaction Table',
  'labels': ['GraphNode', 'database_table']},
 {'uid': 'database_table:t_tochi',
  'name': 'Tochi Transaction Table',
  'labels': ['GraphNode', 'database_table']}]

## 7. Test Relationship Patterns

In [9]:
def explore_relationships(driver, limit=5):
    """Explore relationship patterns in the graph."""
    with driver.session() as session:
        result = session.run("""
            MATCH (a)-[r]->(b)
            RETURN 
                labels(a) as source_labels,
                type(r) as relationship,
                labels(b) as target_labels,
                count(*) as count
            ORDER BY count DESC
            LIMIT $limit
        """, limit=limit).data()
        
        print("=== Common Relationship Patterns ===")
        for item in result:
            source = ":".join(item['source_labels'])
            target = ":".join(item['target_labels'])
            print(f"  ({source})-[{item['relationship']}]->({target}): {item['count']} times")

explore_relationships(driver)

=== Common Relationship Patterns ===
  (GraphNode:function)-[uses_table]->(GraphNode:database_table): 127 times
  (GraphNode:form_group)-[CONTAINS_FIELD]->(GraphNode:form_field): 84 times
  (GraphNode:TabImpact)-[IMPACTS]->(GraphNode:tab): 19 times
  (GraphNode:ProcessFlow)-[BELONGS_TO]->(GraphNode:function): 6 times
  (GraphNode:form)-[HAS_GROUP]->(GraphNode:form_group): 6 times


## 9. Close Connection

## 8. Test Text2Cypher Retriever

Test the Neo4j GraphRAG Text2Cypher retriever with natural language queries.

In [10]:
# Setup LLM - uses your NXCHAT_API configuration
# Make sure OPENAI_API_KEY and optionally OPENAI_API_BASE are set in your .env
import os

# If using custom OpenAI endpoint, set these
if os.getenv("NXCHAT_API"):
    os.environ["OPENAI_API_BASE"] = os.getenv("NXCHAT_API")
if os.getenv("NXCHAT_API_KEY"):
    os.environ["OPENAI_API_KEY"] = os.getenv("NXCHAT_API_KEY")

# Create LLM instance
llm = OpenAILLM(
    model_name="qwen3-235b-a22b",  # or your custom model
    base_url = os.getenv("NXCHAT_API"),
    model_params={
        "temperature": 0.0,  # More deterministic
    }
)

print(llm)

print("✓ LLM configured")


✓ LLM configured


In [11]:
# Create Text2CypherRetriever
# Schema will be auto-fetched from Neo4j when set to None
retriever = Text2CypherRetriever(
    driver=driver,
    llm=llm,
    neo4j_schema=None,  # Will auto-fetch schema from database
    examples=[
        # Optional: Add few-shot examples to improve query generation
        # "Question: How many nodes are there? Cypher: MATCH (n) RETURN count(n) as count",
        # "Question: Show me all forms. Cypher: MATCH (n:Form) RETURN n LIMIT 10",
    ],
)

print("✓ Text2CypherRetriever created successfully")

✓ Text2CypherRetriever created successfully


In [ ]:
# Test with a natural language question
question = "How many Dao"
print(f"Question: {question}\n")
print("Sending to Text2Cypher retriever...")

# Search will:
# 1. Convert the question to Cypher using LLM
# 2. Execute the generated Cypher query
# 3. Return results
result = retriever.search(query_text=question)

print("\n" + "="*60)
print("RESULT:")
print("="*60)
pprint(result)

Question: What determine whether a tab is allowed for switching or not

Sending to Text2Cypher retriever...


Retrying neo4j_graphrag.utils.rate_limit.rate_limit_handler.<locals>.wrapper.<locals>.inner_func in 1.6935625557735834 seconds as it raised RateLimitError: Rate limit exceeded: Error code: 429 - {'error': {'message': 'litellm.RateLimitError: RateLimitError: OpenrouterException - {"error":{"message":"Provider returned error","code":429,"metadata":{"raw":"qwen/qwen3-235b-a22b:free is temporarily rate-limited upstream. Please retry shortly, or add your own key to accumulate your rate limits: https://openrouter.ai/settings/integrations","provider_name":"Venice"}},"user_id":"user_2utBQ5dmAJ26cseZPTtiOxwQMVu"}No fallback model group found for original model_group=qwen3-235b-a22b. Fallbacks=[{\'nxchat-internal\': [\'nxchat-internal-77\']}]. Received Model Group=qwen3-235b-a22b\nAvailable Model Group Fallbacks=None\nError doing the fallback: litellm.RateLimitError: RateLimitError: OpenrouterException - {"error":{"message":"Provider returned error","code":429,"metadata":{"raw":"qwen/qwen3-235b-


RESULT:
RetrieverResult(items=[RetrieverResultItem(content='<Record tab_affected_by=None screen_access_level=None tab_ui_impact=None>', metadata=None), RetrieverResultItem(content="<Record tab_affected_by='tab:basic_information' screen_access_level=None tab_ui_impact='Contract signing date affects order, order date updated'>", metadata=None), RetrieverResultItem(content="<Record tab_affected_by='tab:basic_information' screen_access_level=None tab_ui_impact='System checks project integration data, Tab Order displays new sales office information'>", metadata=None), RetrieverResultItem(content="<Record tab_affected_by='tab:basic_information' screen_access_level=None tab_ui_impact='Tab Order displays project name, construction address updated'>", metadata=None), RetrieverResultItem(content="<Record tab_affected_by='tab:basic_information' screen_access_level=None tab_ui_impact='Tab Order displays correct main/sub customers, display order changed according to new classification'>", metadata

### Understanding the Result Object

The retriever returns a structured result object. Let's extract the key components:

In [14]:
# Inspect the result object structure
print("Result Type:", type(result))
print("\nResult Attributes:")
for attr in dir(result):
    if not attr.startswith('_'):
        print(f"  - {attr}")

# Check if it has common attributes
print("\n" + "="*60)
print("EXTRACTING INFORMATION FROM RESULT")
print("="*60)

# Method 1: Try common attribute names
if hasattr(result, 'items'):
    print("\n✓ Result has 'items' attribute (Pydantic model)")
    print(f"Type: {type(result.items)}")
    print(f"Content: {result.items}")

if hasattr(result, 'content'):
    print("\n✓ Result has 'content' attribute")
    print(f"Content: {result.content}")

# Method 2: Try model_dump if it's a Pydantic model
if hasattr(result, 'model_dump'):
    print("\n✓ Result is a Pydantic model, dumping to dict:")
    result_dict = result.model_dump()
    pprint(result_dict)

Result Type: <class 'neo4j_graphrag.types.RetrieverResult'>

Result Attributes:
  - construct
  - copy
  - dict
  - from_orm
  - items
  - json
  - metadata
  - model_computed_fields
  - model_config
  - model_construct
  - model_copy
  - model_dump
  - model_dump_json
  - model_extra
  - model_fields
  - model_fields_set
  - model_json_schema
  - model_parametrized_name
  - model_post_init
  - model_rebuild
  - model_validate
  - model_validate_json
  - model_validate_strings
  - parse_file
  - parse_obj
  - parse_raw
  - schema
  - schema_json
  - update_forward_refs
  - validate

EXTRACTING INFORMATION FROM RESULT

✓ Result has 'items' attribute (Pydantic model)
Type: <class 'list'>
Content: [RetrieverResultItem(content='<Record tab_affected_by=None screen_access_level=None tab_ui_impact=None>', metadata=None), RetrieverResultItem(content="<Record tab_affected_by='tab:basic_information' screen_access_level=None tab_ui_impact='Contract signing date affects order, order date updated'>

In [16]:
# Extract the answer from the result
def extract_answer_from_result(result):
    """
    Extract the natural language answer from Text2CypherRetriever result.
    
    Returns: (answer, cypher_query, raw_data)
    """
    answer = None
    cypher_query = None
    raw_data = None
    
    # For Pydantic models (neo4j-graphrag v0.5+)
    if hasattr(result, 'model_dump'):
        result_dict = result.model_dump()
        
        # Extract items (query results)
        raw_data = result_dict.get('items', [])
        
        # Extract generated cypher
        metadata = result_dict.get('metadata', {})
        cypher_query = metadata.get('cypher', None)
        
        # The answer needs to be constructed from the data
        # neo4j-graphrag may not always provide a formatted answer
        if raw_data:
            # Format the raw data into a readable answer
            if len(raw_data) == 1 and isinstance(raw_data[0], dict) and len(raw_data[0]) == 1:
                # Single value result (like a count)
                answer = str(list(raw_data[0].values())[0])
            else:
                # Multiple results - format as list
                answer = f"Found {len(raw_data)} results:\n"
                for i, item in enumerate(raw_data[:10], 1):  # Limit to first 10
                    answer += f"\n{i}. {item}"
                if len(raw_data) > 10:
                    answer += f"\n... and {len(raw_data) - 10} more"
        else:
            answer = "No results found"
    
    # For dictionary results
    elif isinstance(result, dict):
        answer = result.get('answer', result.get('response', None))
        cypher_query = result.get('cypher', result.get('query', None))
        raw_data = result.get('data', result.get('items', result.get('results', [])))
    
    # For object with attributes
    else:
        if hasattr(result, 'items'):
            raw_data = result.items
        if hasattr(result, 'metadata'):
            cypher_query = result.metadata.get('cypher') if isinstance(result.metadata, dict) else None
    
    return answer, cypher_query, raw_data

# Extract information
answer, cypher_query, raw_data = extract_answer_from_result(result)

print("="*60)
print("EXTRACTED INFORMATION")
print("="*60)

if cypher_query:
    print("\n📝 Generated Cypher Query:")
    print("-" * 60)
    print(cypher_query)

if answer:
    print("\n💬 Answer:")
    print("-" * 60)
    print(answer)

if raw_data:
    print("\n📊 Raw Data (first 5 items):")
    print("-" * 60)
    pprint(raw_data[:5] if isinstance(raw_data, list) else raw_data)
    if isinstance(raw_data, list) and len(raw_data) > 5:
        print(f"\n... and {len(raw_data) - 5} more items")

EXTRACTED INFORMATION

📝 Generated Cypher Query:
------------------------------------------------------------
MATCH (t:tab)
OPTIONAL MATCH (s:screen) WHERE s.screen_id = t.associated_screen_id
OPTIONAL MATCH (ti:TabImpact)-[:IMPACTS]->(t)
RETURN t.affected_by AS tab_affected_by, s.access_level AS screen_access_level, ti.ui_impact AS tab_ui_impact

💬 Answer:
------------------------------------------------------------
Found 20 results:

1. {'content': '<Record tab_affected_by=None screen_access_level=None tab_ui_impact=None>', 'metadata': None}
2. {'content': "<Record tab_affected_by='tab:basic_information' screen_access_level=None tab_ui_impact='Contract signing date affects order, order date updated'>", 'metadata': None}
3. {'content': "<Record tab_affected_by='tab:basic_information' screen_access_level=None tab_ui_impact='System checks project integration data, Tab Order displays new sales office information'>", 'metadata': None}
4. {'content': "<Record tab_affected_by='tab:basic_inf

### Generate Natural Language Answer

Use the LLM to convert the query results into a natural language answer:

In [17]:
def generate_answer_from_results(question, cypher_query, raw_data, llm):
    """
    Use the LLM to generate a natural language answer from query results.
    """
    # Format the data for the prompt
    data_str = json.dumps(raw_data, indent=2, default=str)
    
    # Create a prompt for answer generation
    prompt = f"""Based on the following information, provide a clear and concise answer to the user's question.

Question: {question}

Cypher Query Executed:
{cypher_query}

Query Results:
{data_str}

Please provide a natural language answer that directly addresses the question. Be specific and use the data provided."""

    # Generate the answer
    response = llm.invoke(prompt)
    
    # Extract text from response
    if hasattr(response, 'content'):
        return response.content
    elif isinstance(response, dict):
        return response.get('content', str(response))
    else:
        return str(response)

# Generate a natural language answer
if raw_data and cypher_query:
    print("Generating natural language answer...")
    nl_answer = generate_answer_from_results(question, cypher_query, raw_data, llm)
    
    print("\n" + "="*60)
    print("🤖 GENERATED NATURAL LANGUAGE ANSWER")
    print("="*60)
    print(nl_answer)
else:
    print("⚠ Cannot generate answer - missing query results or cypher query")

Generating natural language answer...

🤖 GENERATED NATURAL LANGUAGE ANSWER
Based on the query results provided, tab switching permissions are determined by dependencies on the 'basic_information' tab. 

The data shows that all tabs (except one null record) have "tab_affected_by='tab:basic_information'", indicating that the basic information tab serves as a prerequisite for other tabs. The screen_access_level field is consistently null across all records, showing that access levels are not the determining factor.

The tab_ui_impact field reveals specific dependency relationships:
- Tabs require certain data elements from the basic information tab to function properly
- Examples include: "Contract signing date affects order", "Deleted customer not displayed in Tab Order", and "Contract conclusion date affects tax calculations"
- These dependencies create validation requirements that must be satisfied before switching

Essentially, a tab is only allowed for switching when all required dat

### Complete Pipeline: Question → Cypher → Results → Answer

Putting it all together in a reusable function:

In [ ]:
def text_to_cypher_pipeline(question, retriever, llm):
    """
    Complete pipeline: Question → Cypher → Execute → Generate Answer
    
    Returns: dict with all intermediate results
    """
    print(f"\n{'='*60}")
    print(f"Question: {question}")
    print('='*60)
    
    # Step 1: Use retriever to generate and execute Cypher
    print("\n[1/3] Generating Cypher query...")
    result = retriever.search(query_text=question)
    
    # Step 2: Extract results
    print("[2/3] Extracting query results...")
    answer, cypher_query, raw_data = extract_answer_from_result(result)
    
    if cypher_query:
        print(f"✓ Generated Cypher:\n{cypher_query}\n")
    
    if raw_data:
        result_count = len(raw_data) if isinstance(raw_data, list) else 1
        print(f"✓ Retrieved {result_count} result(s)")
    
    # Step 3: Generate natural language answer
    print("[3/3] Generating natural language answer...")
    if raw_data and cypher_query:
        nl_answer = generate_answer_from_results(question, cypher_query, raw_data, llm)
    else:
        nl_answer = "Unable to generate answer from the query results."
    
    print("\n" + "="*60)
    print("🎯 FINAL ANSWER")
    print("="*60)
    print(nl_answer)
    print("="*60 + "\n")
    
    return {
        'question': question,
        'cypher_query': cypher_query,
        'raw_data': raw_data,
        'answer': nl_answer,
        'result_object': result
    }

# Test the complete pipeline
test_question = "How many different types of nodes exist in the database?"
pipeline_result = text_to_cypher_pipeline(test_question, retriever, llm)

In [ ]:
# Test with multiple questions using the complete pipeline
test_questions = [
    "What are all the relationship types in the graph?",
    "Show me 3 examples of GraphNode entities",
    "How many total relationships are there?",
    "What properties do GraphNode entities have?",
]

print("\n" + "🔄 TESTING MULTIPLE QUESTIONS" + "\n")

results = []
for q in test_questions:
    try:
        result = text_to_cypher_pipeline(q, retriever, llm)
        results.append(result)
    except Exception as e:
        print(f"❌ Error processing question: {e}")
        import traceback
        traceback.print_exc()

print(f"\n✅ Successfully processed {len(results)}/{len(test_questions)} questions")

In [32]:
# Extract key information from the result
def display_retriever_result(result):
    """Display the Text2Cypher retriever result in a readable format."""
    print("\n" + "="*60)
    print("TEXT2CYPHER RESULT BREAKDOWN")
    print("="*60)
    
    # Try to extract generated Cypher query
    cypher_query = None
    if hasattr(result, 'metadata') and result.metadata:
        cypher_query = result.metadata.get('cypher', None)
    elif hasattr(result, 'query'):
        cypher_query = result.query
    elif isinstance(result, dict):
        cypher_query = result.get('cypher', result.get('query', result.get('generated_cypher', None)))
    
    if cypher_query:
        print("\n📝 Generated Cypher Query:")
        print("-" * 60)
        print(cypher_query)
    
    # Try to extract answer/response
    answer = None
    if hasattr(result, 'answer'):
        answer = result.answer
    elif hasattr(result, 'response'):
        answer = result.response
    elif isinstance(result, dict):
        answer = result.get('answer', result.get('response', result.get('text', None)))
    
    if answer:
        print("\n💬 Answer:")
        print("-" * 60)
        print(answer)
    
    # Try to extract raw data
    data = None
    if hasattr(result, 'data'):
        data = result.data
    elif hasattr(result, 'results'):
        data = result.results
    elif isinstance(result, dict):
        data = result.get('data', result.get('results', result.get('records', None)))
    
    if data:
        print("\n📊 Raw Data:")
        print("-" * 60)
        pprint(data)
    
    print("\n" + "="*60)

display_retriever_result(result)


TEXT2CYPHER RESULT BREAKDOWN

📝 Generated Cypher Query:
------------------------------------------------------------
MATCH (f:function) RETURN f.uid, f.name, f.description



### Try More Questions

Test the retriever with different types of questions:

In [ ]:
# Test different questions
test_questions = [
    "How many nodes are in the database?",
    "Show me 5 GraphNode entities",
    "What relationships exist in the graph?",
    "Find all forms in the system",
]

for i, q in enumerate(test_questions, 1):
    print(f"\n{'='*60}")
    print(f"Question {i}: {q}")
    print('='*60)
    
    try:
        result = retriever.search(query_text=q)
        display_retriever_result(result)
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()

In [ ]:
# Close the driver when done
driver.close()
print("✓ Connection closed")